<a href="https://colab.research.google.com/github/jcdevaney/pyAMPACTtutorials/blob/main/01-pyAMPACT_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>pyAMPACT Introduction</h1>

In [1]:
!git clone https://github.com/jcdevaney/pyAMPACTtutorials.git
from IPython.utils import io
print('Importing libraries...')
with io.capture_output() as captured:
    !pip install --upgrade pandas
    !pip install -i https://test.pypi.org/simple/ --no-deps pyampact
    import pyampact
import pandas as pd
import numpy as np
import librosa

fatal: destination path 'pyAMPACTtutorials' already exists and is not an empty directory.
Importing libraries...


Load audio with librosa and symbolic files with pyAMPACT's `Score` function. Audio files can be any format supported by librosa (wav, mp3, flac, ogg, etc.) and symbolic files can be any format supported by music21 (kern, xml, mei, midi, abc, etc.).

In [2]:
audio_file = '/content/pyAMPACTtutorials/test_files/Mozart_K179_seg.wav'
midi_file = '/content/pyAMPACTtutorials/test_files/Mozart_K179_seg.krn'

y, original_sr = librosa.load(audio_file)

piece = pyampact.Score(midi_file)

Run the alignment with pyampact's `run_alignment` function.



In [3]:
# You can specify the target sample rate (target_sr), window size in
# milliseconds (win_ms), the hop size (hop_length) as well as arguments
# that dictate the way that the symbolic file is represented as a
# spectrogram-like mask: the bin width of each note/harmonic (width)
# and the number of harmonics (n_harm).
target_sr = 4000
win_ms = 100
hop_length = 32
width = 3
n_harm = 3

# Visualize the alignment with True flag

res, dtw, spec, nmat = pyampact.run_alignment(
    y, original_sr, piece, piece.nmats(), width, target_sr, n_harm, win_ms, hop_length)

Estimate performance paramters.

THIS IS NOT WORKING

In [5]:
# Calculate an instananeous frequency-gram of the audio signal
# What are the f0_valuues here?
#freqs, times, mags, f0_values, mags_mat = pyampact.ifgram(audiofile=audio_file, tsr=target_sr, win_ms=win_ms)
#mags_db = librosa.amplitude_to_db(mags, ref=np.max)

# Estimate F0 values
# This is just calculating a single F0 for each frame, but separate F0s need to be calculated
# for each polyphonic note (as per https://github.com/jcdevaney/AMPACT-working/blob/main/f0EstWeightedSumSpec.m)
# A spectrum also need to be calculated and return for estimating timbral parameters
# The NMAT looped through and the onset, offset, and MIDI note passed to the calculate_f0_est function
# [f0{v}{loc}, pwr{v}{loc}, t{v}{loc}, M{v}{loc}, xf{v}{loc}] = f0EstWeightedSumSpec(audiofile, ons(loc), offs(loc), nmat{v}(n,4));
# In the original MATLAB code there was a separate NMAT for each part/voice (v) this can be done by creating a dictionary
# out of the nmat
#   nmat_dict = piece.nmats()
#   nmat_dict['Part-1']
f0_values, sig_pwr = pyampact.calculate_f0_est(audio_file, hop_length, win_ms, target_sr)
sig_pwr = mags ** 2 # power of signal, magnitude/amplitude squared
# Prune NaN and zero values from estimated F0 and power
f0_values = f0_values[~np.isnan(f0_values)]
sig_pwr = sig_pwr[sig_pwr != 0]


NameError: name 'mags' is not defined

Compile estimated performance parameters

In [ ]:
pyampact.data_compilation(f0_values, sig_pwr, mags_mat, nmat, target_sr, hop_length, y, audio_file)

Export estimated performance parameters to a kern or MEI file

In [ ]:
piece.toKern()

In [ ]:
piece.toMEI()